In [ ]:
%load_ext autoreload
%autoreload 2
import waffles
import numpy as np
import json
import shutil 
from tqdm import tqdm

from waffles.input_output.hdf5_structured import load_structured_waveformset
from waffles.data_classes.Waveform import Waveform
from waffles.data_classes.WaveformSet import WaveformSet
from waffles.data_classes.BasicWfAna import BasicWfAna
from waffles.data_classes.IPDict import IPDict
from waffles.data_classes.UniqueChannel import UniqueChannel
from waffles.data_classes.ChannelWsGrid import ChannelWsGrid
from waffles.utils.baseline.baseline import SBaseline
from waffles.np02_utils.AutoMap import generate_ChannelMap, dict_uniqch_to_module, dict_module_to_uniqch, ordered_channels_cathode, ordered_channels_membrane, strUch, ordered_modules_cathode, ordered_modules_membrane
from waffles.np02_utils.PlotUtils import np02_gen_grids, plot_grid, plot_detectors, genhist, fithist, runBasicWfAnaNP02, ch_read_calib
from waffles.np02_utils.load_utils import open_processed

In [ ]:
mask_from_run = {
40803: 1,
40804: 2,
40805: 4,
40807: 8,
40808: 16,
40809: 32,
40810: 64,
}


nwaveforms = 80000
run = list(mask_from_run.keys())[4]
dettype = "membrane"
dettype = "cathode"
det = "VD_Cathode_PDS" if dettype == "cathode" else "VD_Membrane_PDS"
endpoint = 106 if dettype == "cathode" else 107
listofch = ordered_channels_cathode if endpoint==106 else ordered_channels_membrane
validchannels = {}
validchannels[106] = ordered_channels_cathode 
validchannels[107] = ordered_channels_membrane

# Way to low... keep scrollng
dletter = dettype.upper()[0] # C or M...
group1 = [ f"{dletter}{detnum}({chnum})" for detnum in range(1, 3) for chnum in range(1,3) ]
group2 = [ f"{dletter}{detnum}({chnum})" for detnum in range(3, 5) for chnum in range(1,3) ]
group3 = [ f"{dletter}{detnum}({chnum})" for detnum in range(5, 7) for chnum in range(1,3) ]
group4 = [ f"{dletter}{detnum}({chnum})" for detnum in range(7, 9) for chnum in range(1,3) ]

In [ ]:
wfset_full = open_processed(run,dettype=dettype, datadir="/eos/experiment/neutplatform/protodune/experiments/ProtoDUNE-VD/commissioning/", nwaveforms=nwaveforms)

In [ ]:
from collections import Counter
timestamps = sorted([ wf.timestamp for wf in wfset_full.waveforms ])
c = Counter(timestamps)
print(f"Total number of timestamps: {len(c)}")
matchtimestamps = [ k for k in c if c[k] >= len(list(wfset_full.available_channels[run])) ]
print(f"Remaining timestamps: {len(matchtimestamps)}")
if len(c) == len(matchtimestamps):
    matchtimestamps = [] # no need for filtering

In [ ]:
def get_external(waveform: Waveform, validtimes = [], validchannels = []) -> bool:
    # if np.any((waveform.adcs[0:500] > 16000) | (waveform.adcs[0:500] < 100) ):
    #     return False

    if waveform.channel not in validchannels:
        return False

    if len(validtimes) and waveform.timestamp  not in validtimes:
        return False

    
    return True

matchingtimes = []
print(wfset_full)
wfset= WaveformSet.from_filtered_WaveformSet(wfset_full, get_external, matchtimestamps, validchannels[endpoint], show_progress=True)
print(wfset)

In [ ]:
# runBasicWfAnaNP02(wfset_full, int_ul=700, onlyoptimal=False)

In [ ]:
histargs = dict(    
    # doprocess = False,
    dofit = False,
    variable = 'integral',
    # show_progress = True,
    # configyaml="ch_snr_parameters.yaml",
    normalize_hist=False,
    histautorange=True,
    onlyoptimal=True
)

detector=group1+group2
plot_detectors(wfset, detector, plot_function=fithist, html=None, wf_func=histargs)
detector=group3+group4
plot_detectors(wfset, detector, plot_function=fithist, html=None, wf_func=histargs)


In [ ]:
def onlyvalid(waveform: Waveform) -> bool:
    v = waveform.analyses['std'].result['integral']
    if np.isnan(v):
        return False
    return True
wfset_filt = WaveformSet.from_filtered_WaveformSet(wfset, onlyvalid, show_progress=True)

In [ ]:
argsheat = dict(
    mode="heatmap",
    analysis_label="std",
    adc_range_above_baseline=10000,
    adc_range_below_baseline=-500,
    adc_bins=125,
    time_bins=wfset_full.points_per_wf//2,
    filtering=4,
    share_y_scale=False,
    share_x_scale=True,
    wfs_per_axes=5000,
    zlog=True
)
detector=group1+group2
plot_detectors(wfset, detector, **argsheat)

detector=group3+group4
plot_detectors(wfset, detector, **argsheat)



In [ ]:
wfsetch = ChannelWsGrid.clusterize_waveform_set(wfset)

In [ ]:
calib_dict = ch_read_calib()

In [ ]:
import pandas as pd
import os

CSV_PATH = "detector_responses.csv"

def upsert_row(mask, module, value):
    # 1. Load existing CSV or create an empty DataFrame
    if os.path.exists(CSV_PATH):
        df = pd.read_csv(CSV_PATH)
    else:
        df = pd.DataFrame(columns=["mask", "module", "value"])

    # 2. Check if this (mask, module) already exists
    match = (df["mask"] == mask) & (df["module"] == module)

    if match.any():
        # Update existing row
        df.loc[match, "value"] = value
    else:
        # Insert new row
        df = pd.concat(
            [df if not df.empty else None, pd.DataFrame([{"mask": mask, "module": module, "value": value}])],
            ignore_index=True
        )

    # 3. Save back to CSV
    df.to_csv(CSV_PATH, index=False)


In [ ]:
import matplotlib.pyplot as plt
modules_npes = {}
for ep, wfch in wfsetch.items():
    for ch in listofch:
        full_ch_name = dict_uniqch_to_module[strUch(ep,ch)]
        module_name = full_ch_name[:2]
        if module_name not in modules_npes:
            modules_npes[module_name] = []
        if ch not in wfch.keys() or full_ch_name in ["M2(1)", "C3(2)"]:
            npes = np.nan
        else:
            wfs = wfch[ch]
            specharge = calib_dict[ep][ch]['Gain']
            charges = np.array([ wf.analyses['std'].result['integral'] for wf in wfs.waveforms ])
            charges = charges[~np.isnan(charges)]
            charges = charges[charges>0]
            if len(charges):
                clow, cup = np.quantile(charges, [0.02, 0.98])
                if len(charges)<5:
                    clow, cup = np.array([np.min(charges), np.max(charges)])
                npes = charges[ (charges >= clow) & (charges <= cup)].mean()/specharge
            else: npes = np.nan
        # print(module_name, dict_uniqch_to_module[strUch(ep,ch)], npes)
        modules_npes[module_name] += [npes]

for m, npe in modules_npes.items():
    print(mask_from_run[run], m, npe, np.nanmean(npe) if len(npe) else np.nan)
    upsert_row(mask_from_run[run], m, np.nanmean(npe) if len(npe) else np.nan)

In [ ]:
df = pd.read_csv("detector_responses.csv")
df

In [ ]:
df2 = df.pivot(index="mask", columns="module", values="value")

In [ ]:
df2.reset_index()

In [ ]:
import matplotlib.pyplot as plt
wfs = ChannelWsGrid.clusterize_waveform_set(wfset_filt)[endpoint][43].waveforms

In [ ]:
# charges = [ w.analyses['std'].result['integral'] for w in wfs]
# plt.hist(charges, bins=500);
a+=1
plt.plot(wfs[a].adcs - wfs[a].analyses['std'].result['baseline'])
print(wfs[a].analyses['std'].result['integral'])
